# 🚀 Data Preparation Exercise (SageMaker + Redshift)

**Purpose:** Build a *stable*, **reproducible**, and **config-driven** data prep flow you can promote to production.

> This is an **exercise** version of the example. **You choose** the data source (Redshift or S3 Parquet) and the **transformations**. Update the cells marked with `# <- TODO ✏️`.

## 📦 What You’ll Deliver
- A config-first **data loader** using `load_data()` from `data_io.py` *(or demo data fallback)*  
- Your own **cleaning & feature engineering** rules (deterministic)  
- **Leakage-aware** train/val/test splits → `splits.json`  
- Export the **processed dataset** → Parquet  

## 🧰 Prerequisites
- Python 3.9+
- Packages: `pandas`, `numpy`, `pyarrow`, `scikit-learn`, `sqlalchemy`, `redshift_connector`, `s3fs` (and optionally `mlflow`)
- A `data_io.py` next to this notebook containing a `load_data(source, uri, sql, redshift_kwargs)` function.


# If running on a fresh environment (SageMaker usually has these), uncomment as needed
# %pip install pandas numpy pyarrow scikit-learn sqlalchemy redshift_connector s3fs mlflow

In [ ]:
import os, sys, json, time, hashlib, platform, random
from datetime import datetime
import numpy as np
import pandas as pd

# ♻️ Reproducibility & Environment Capture
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

RUN_TS = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RUN_ID = hashlib.sha1(f"{RUN_TS}-{SEED}".encode()).hexdigest()[:10]

ARTIFACT_DIR = os.environ.get("ARTIFACT_DIR", f"artifacts/run_{RUN_TS}_{RUN_ID}")
os.makedirs(ARTIFACT_DIR, exist_ok=True)

env_info = {
    "python": sys.version,
    "platform": platform.platform(),
    "timestamp_utc": RUN_TS,
    "seed": SEED,
    "packages": {
        "pandas": pd.__version__,
        "numpy": np.__version__,
    },
}
with open(os.path.join(ARTIFACT_DIR, "env_info.json"), "w") as f:
    json.dump(env_info, f, indent=2)

env_info

## ⚙️ Configuration
Single source of truth for inputs, outputs, and behavior. Switch **source** between `"redshift"` and `"parquet"` here. Fill in your S3 path or SQL.

> Edit the **TODO** fields to point to your data and tweak processing choices.


In [ ]:
from pathlib import Path

CONFIG = {
    "data": {
        # choose 'redshift' or 'parquet'
        "source": os.environ.get("SOURCE", "parquet"),  # <- TODO ✏️ 'parquet' or 'redshift'
        # parquet URI if using S3 parquet (supports wildcards)
        "parquet_uri": os.environ.get("PARQUET_URI", "s3://your-bucket/path/to/data/*.parquet"),  # <- TODO ✏️ your S3 path
        # SQL for Redshift (must be deterministic if you sample! include ORDER BY)
        "sql": os.environ.get("SQL", "SELECT * FROM your_schema.your_table ORDER BY id"),  # <- TODO ✏️ your query
        # Redshift connection kwargs (fill via env vars or IAM role inside SageMaker)
        "redshift_kwargs": {
            "host": os.environ.get("REDSHIFT_HOST", "example.your-cluster.redshift.amazonaws.com"),  # <- TODO ✏️
            "database": os.environ.get("REDSHIFT_DB", "dev"),  # <- TODO ✏️
            "user": os.environ.get("REDSHIFT_USER", "username"),  # <- TODO ✏️
            "password": os.environ.get("REDSHIFT_PASSWORD", "password"),  # <- TODO ✏️ (or use IAM)
            "port": int(os.environ.get("REDSHIFT_PORT", "5439")),
        },
        # Optional: limit rows deterministically for dev runs (None = no limit)
        "row_limit": int(os.environ.get("ROW_LIMIT", "50000")),  # <- TODO ✏️ adjust for quick iteration
    },
    "columns": {
        # Define your target column name (classification or regression)
        "target": os.environ.get("TARGET", "churned"),  # <- TODO ✏️
        # Optional primary key for splits and deterministic joins
        "primary_key": os.environ.get("PRIMARY_KEY", "customer_id"),  # <- TODO ✏️
        # Optional timestamp column for time-based split
        "timestamp": os.environ.get("TS_COL", ""),  # <- TODO ✏️ e.g., event_time
    },
    "processing": {
        "stratify_splits": True,       # set False for regression  # <- TODO ✏️
        "test_size": 0.2,              # <- TODO ✏️
        "val_size": 0.1,               # <- TODO ✏️
        "dropna_threshold_ratio": 0.95,  # drop columns with >5% missing if needed  # <- TODO ✏️
        "cap_outliers_iqr": True,        # <- TODO ✏️
        "normalize_categoricals": True,  # <- TODO ✏️
    },
    "output": {
        "artifact_dir": ARTIFACT_DIR,
        "processed_parquet_path": str(Path(ARTIFACT_DIR) / "processed" / "dataset.parquet"),
        "feature_schema_path": str(Path(ARTIFACT_DIR) / "feature_schema.json"),
        "splits_path": str(Path(ARTIFACT_DIR) / "splits.json"),
    },
    "mlflow": {
        "enabled": False,  # <- TODO ✏️ enable if you want lineage
        "tracking_uri": os.environ.get("MLFLOW_TRACKING_URI", ""),
        "experiment_name": os.environ.get("MLFLOW_EXPERIMENT", "data-prep-exercise"),
    }
}

CONFIG

## 📥 Load Data (Redshift or S3 Parquet)

We will try to import `load_data()` from `data_io.py`. If not found, we fall back to a **synthetic demo dataset** so the rest of the exercise remains runnable.


In [ ]:
# Try to import the provided load_data function from data_io.py
load_data = None
try:
    from data_io import load_data  # expects file next to this notebook
except Exception as e:
    print("⚠️ Could not import `load_data` from data_io.py. Using synthetic demo data. Error:", repr(e))

def _demo_dataset(n=3000, seed=SEED):
    rng = np.random.default_rng(seed)
    df = pd.DataFrame({
        "customer_id": np.arange(1, n+1),
        "age": rng.integers(18, 90, size=n),
        "tenure_months": rng.integers(0, 120, size=n),
        "monthly_charges": rng.normal(45, 15, size=n).round(2),
        "contract_type": rng.choice(["month-to-month", "one-year", "two-year"], size=n, p=[0.6, 0.25, 0.15]),
        "country": rng.choice(["PT","ES","FR","DE"], size=n, p=[0.5,0.2,0.2,0.1]),
        "signup_ts": pd.to_datetime("2022-01-01") + pd.to_timedelta(rng.integers(0, 900, size=n), unit="D"),
        "churned": rng.choice([0,1], size=n, p=[0.78, 0.22]).astype(int),
    })
    # anomalies to clean
    df.loc[rng.choice(df.index, 15, replace=False), "monthly_charges"] = -1.0
    df.loc[rng.choice(df.index, 25, replace=False), "age"] = None
    return df

if load_data:
    source = CONFIG["data"]["source"]
    uri = CONFIG["data"]["parquet_uri"]
    sql = CONFIG["data"]["sql"]
    rs_kwargs = CONFIG["data"]["redshift_kwargs"]
    print(f"Loading data via data_io.load_data(source={source!r}) ...")
    df_raw = load_data(source=source, uri=uri, sql=sql, redshift_kwargs=rs_kwargs)
else:
    print("Using synthetic dataset for demonstration.")
    df_raw = _demo_dataset(n=CONFIG["data"]["row_limit"] or 3000)

# Optional row limit for dev runs
row_limit = CONFIG["data"]["row_limit"]
if row_limit and len(df_raw) > row_limit:
    pk = CONFIG["columns"]["primary_key"]
    if pk in df_raw.columns:
        df_raw = df_raw.sort_values(pk).head(row_limit).reset_index(drop=True)
    else:
        df_raw = df_raw.sample(n=row_limit, random_state=SEED).reset_index(drop=True)

df_raw.head(), df_raw.shape

## 🔎 Quick Profile
Lightweight overview—scan types, nulls, and rough completeness.


In [ ]:
pd.DataFrame({
    "column": df_raw.columns,
    "dtype": df_raw.dtypes.astype(str).values,
    "nulls": [df_raw[c].isna().sum() for c in df_raw.columns],
    "non_nulls": [df_raw[c].notna().sum() for c in df_raw.columns],
}).head(40)

## 🧾 Feature Schema (Draft)
Define **types, nullability, min/max for numerics**, and sample categories for strings. Export to JSON.


In [ ]:
import json
from typing import Any, Dict

target_col = CONFIG["columns"]["target"]
primary_key = CONFIG["columns"]["primary_key"]

def infer_basic_schema(df: pd.DataFrame) -> Dict[str, Any]:
    schema = {}
    for c in df.columns:
        col_dtype = str(df[c].dtype)
        col = { "dtype": col_dtype, "nullable": bool(df[c].isna().any()) }
        if pd.api.types.is_numeric_dtype(df[c]):
            finite_vals = pd.to_numeric(df[c], errors="coerce").replace([np.inf, -np.inf], np.nan).dropna()
            if len(finite_vals):
                col["min"] = float(finite_vals.min())
                col["max"] = float(finite_vals.max())
        else:
            col["example_values"] = df[c].dropna().astype(str).unique()[:20].tolist()
        schema[c] = col
    schema_meta = {
        "_meta": {
            "target": target_col if target_col in df.columns else None,
            "primary_key": primary_key if primary_key in df.columns else None,
            "generated_at": RUN_TS,
            "seed": SEED,
        },
        "columns": schema
    }
    return schema_meta

feature_schema = infer_basic_schema(df_raw)

schema_path = CONFIG["output"]["feature_schema_path"]
os.makedirs(Path(schema_path).parent, exist_ok=True)
with open(schema_path, "w") as f:
    json.dump(feature_schema, f, indent=2)

schema_path

## 🧼 Cleaning (YOU decide)
Update the block below to **codify** your rules. Keep them deterministic and versioned.

Suggestions (edit as needed):
- Type coercions for IDs
- Invalid value fixes (e.g., negative price → NaN)
- Missing value strategies (numeric/categorical)
- Optional outlier capping (IQR)


In [ ]:
df = df_raw.copy()

# IDs as strings (preserve leading zeros)
if primary_key in df.columns:
    df[primary_key] = df[primary_key].astype(str)  # <- TODO ✏️ confirm your PK name

# Example rule: fix invalid negatives in 'monthly_charges'
if "monthly_charges" in df.columns:
    df.loc[df["monthly_charges"] < 0, "monthly_charges"] = np.nan  # <- TODO ✏️ adapt to your domain

# Build column type lists (excluding target)
num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c]) and c != target_col]
cat_cols = [c for c in df.columns if (not pd.api.types.is_numeric_dtype(df[c])) and c not in [target_col]]

# Missing values — simple, deterministic strategy (EDIT if you need smarter imputers)
for c in num_cols:
    median_val = df[c].median()
    df[c] = df[c].fillna(median_val)  # <- TODO ✏️ try mean/constant or KNN later

for c in cat_cols:
    df[c] = df[c].fillna("__MISSING__")  # <- TODO ✏️ choose a sentinel

# Normalize categoricals (lowercase/trim) for consistency
if bool(CONFIG["processing"]["normalize_categoricals"]):
    for c in cat_cols:
        df[c] = df[c].astype(str).str.strip().str.lower()

# Optional IQR capping for outliers
if bool(CONFIG["processing"]["cap_outliers_iqr"]):
    for c in num_cols:
        q1, q3 = np.percentile(df[c], [25, 75])
        iqr = q3 - q1
        lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
        df[c] = np.clip(df[c], lower, upper)  # <- TODO ✏️ choose columns to cap

df.head()

## 🧩 Feature Engineering (YOU decide)
Add domain features and lightweight encodings here. Defer heavy encoders to training.

Examples:
- Ratios/interactions
- Date parts from timestamps
- Frequency or target-aware encodings (deterministic)


In [ ]:
# Example engineered features (EDIT/ADD/REMOVE)
ts_col = CONFIG["columns"].get("timestamp")  # <- TODO ✏️ set in config if you have time
if ts_col and ts_col in df.columns:
    ts = pd.to_datetime(df[ts_col], errors="coerce")
    df["ts_year"] = ts.dt.year
    df["ts_month"] = ts.dt.month
    df["ts_dow"] = ts.dt.dayofweek

# Example: lifetime value proxy
if set(["tenure_months","monthly_charges"]).issubset(df.columns):
    df["est_lifetime_value"] = (df["tenure_months"] * df["monthly_charges"]).round(2)  # <- TODO ✏️ your formula

# Simple frequency encoding for categoricals (kept numeric)
for c in [c for c in df.columns if (c not in [target_col]) and (df[c].dtype == object or isinstance(df[c].dtype, pd.StringDtype))]:
    freq = df[c].value_counts(normalize=True)
    df[f"{c}__freq"] = df[c].map(freq).astype(float)

df.head()

## ✅ Lightweight Validation
Add simple checks before export. For production, consider Great Expectations or pandera.


In [ ]:
checks = []

# 1) Target checks
if target_col in df.columns and CONFIG["processing"]["stratify_splits"]:
    uniq = pd.Series(df[target_col]).dropna().unique()
    is_binary = set(uniq) <= {0,1}
    checks.append({"check": "target_binary_if_classif", "passed": bool(is_binary), "unique": uniq.tolist()})

# 2) Primary key not null
if primary_key in df.columns:
    pk_nulls = int(df[primary_key].isna().sum())
    checks.append({"check": "no_null_primary_key", "passed": pk_nulls == 0, "null_count": pk_nulls})

# 3) Optional: drop columns with too many nulls (ratio threshold)
thr = float(CONFIG["processing"]["dropna_threshold_ratio"])
null_ratio = df.isna().mean().to_dict()
checks.append({"check": "null_ratio_snapshot", "passed": True, "details": {k: round(v,4) for k,v in null_ratio.items()}})

pd.DataFrame(checks)

## ✂️ Train / Validation / Test Split
- Deterministic with fixed `random_state`
- Stratified if classification
- Optionally **time-based** using your timestamp column


In [ ]:
from sklearn.model_selection import train_test_split

random_state = SEED
stratify = None
if CONFIG["processing"]["stratify_splits"] and (target_col in df.columns):
    stratify = df[target_col]

ts_col = CONFIG["columns"].get("timestamp", "")
if ts_col and ts_col in df.columns:
    ts = pd.to_datetime(df[ts_col], errors="coerce")
    # 80/20 split by time, then split 20% into val/test by ratio
    cutoff = ts.quantile(1.0 - CONFIG["processing"]["test_size"])
    train_val_df = df[ts < cutoff].copy()
    test_df = df[ts >= cutoff].copy()
else:
    train_val_df, test_df = train_test_split(
        df,
        test_size=CONFIG["processing"]["test_size"],
        random_state=random_state,
        stratify=stratify
    )

# Second split: train vs val
stratify_train_val = train_val_df[target_col] if (stratify is not None and target_col in train_val_df.columns) else None
val_ratio = CONFIG["processing"]["val_size"] / (1.0 - CONFIG["processing"]["test_size"])
train_df, val_df = train_test_split(
    train_val_df,
    test_size=val_ratio,
    random_state=random_state,
    stratify=stratify_train_val
)

len(train_df), len(val_df), len(test_df)

## 💾 Write Artifacts
- **Processed dataset** (Parquet)
- **Feature schema** (`feature_schema.json`)
- **Splits** (`splits.json`) with IDs for deterministic reuse


In [ ]:
from pathlib import Path
out_path = Path(CONFIG["output"]["processed_parquet_path"])
out_path.parent.mkdir(parents=True, exist_ok=True)

# Save full processed dataset
df.to_parquet(out_path, index=False)

# Save split IDs by primary key (preferred) or DataFrame indices
splits = {}
pk = primary_key if primary_key in df.columns else None
def ids_of(subdf):
    if pk:
        return subdf[pk].tolist()
    else:
        return subdf.index.tolist()

splits = {
    "meta": {
        "seed": SEED,
        "created_at": RUN_TS,
        "primary_key": pk,
        "target": target_col if target_col in df.columns else None,
        "source": CONFIG["data"]["source"],
    },
    "train_ids": ids_of(train_df),
    "val_ids": ids_of(val_df),
    "test_ids": ids_of(test_df),
}

with open(CONFIG["output"]["splits_path"], "w") as f:
    json.dump(splits, f, indent=2)

{
    "processed_parquet": str(out_path),
    "feature_schema": CONFIG["output"]["feature_schema_path"],
    "splits": CONFIG["output"]["splits_path"],
}

## 📈 (Optional) MLflow Trace
Enable by setting `CONFIG["mlflow"]["enabled"] = True`.


In [ ]:
if CONFIG["mlflow"]["enabled"]:
    import mlflow
    mlflow.set_tracking_uri(CONFIG["mlflow"]["tracking_uri"] or "file://" + str(Path(ARTIFACT_DIR).absolute()))
    mlflow.set_experiment(CONFIG["mlflow"]["experiment_name"])

    with mlflow.start_run(run_name=f"data-prep-exercise-{RUN_TS}") as run:
        mlflow.log_params({
            "seed": SEED,
            "source": CONFIG["data"]["source"],
            "row_limit": CONFIG["data"]["row_limit"],
            "stratify": CONFIG["processing"]["stratify_splits"],
        })
        mlflow.log_artifact(CONFIG["output"]["feature_schema_path"])
        mlflow.log_artifact(CONFIG["output"]["splits_path"])
        # logging the whole parquet can be large; consider sampling or schema-only
        # mlflow.log_artifact(CONFIG["output"]["processed_parquet_path"])
        print("MLflow run:", run.info.run_id)